In [15]:
import xarray
from datetime import datetime, timedelta
from netCDF4 import date2num

In [36]:
d0 = datetime(1960, 1, 28)
t = [d0 + i * timedelta(hours=1) for i in range(1000)] 
data = [float(i) for i in list(range(len(t)))]

In [37]:
units = "hours since {:%Y-%m-%d %H:%M:%S}".format(t[0])
units

'hours since 1960-01-28 00:00:00'

In [38]:

ds = xarray.Dataset({
        "test": (("time",), data, {"description": "Test"})
    },
    coords={"time": ("time", t),
           #"calendar": "365_day"
           })

In [39]:
ds

<xarray.Dataset>
Dimensions:  (time: 1000)
Coordinates:
  * time     (time) datetime64[ns] 1960-01-28 1960-01-28T01:00:00 ...
Data variables:
    test     (time) float64 0.0 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0 ...

In [40]:
import numpy as np
# remove Feb29 if appeared
ds["test"][(ds["time.month"] == 2) & (ds["time.day"] == 29)] = np.nan
ds = ds.dropna(dim="time", how="all")


In [41]:
ds.to_netcdf("test.nc", encoding={"time": {"calendar": "365_day"}})

In [21]:
%%bash
ncdump -h test.nc

netcdf test {
dimensions:
	time = 100 ;
variables:
	int64 test(time) ;
		test:description = "Test" ;
	double time(time) ;
		time:units = "hours since 1960-01-28 00:00:00" ;
		time:calendar = "365_day" ;
}


In [45]:
sum((ds["time.month"] == 2) & (ds["time.day"] == 29))

<xarray.DataArray ()>
array(0)

In [24]:
ds2 = ds.sel[ds["time"] != datetime]

TypeError: 'method' object is not subscriptable